In [1]:
import itertools
import numpy as np

In [4]:
# 重み付きユニットの一般化(step 2)

# しきい値を重みの一つとみなすと、計算式を簡略化できる。
# 入力: x, 重み: w, しきい値: tの場合、
# x * w と t の比較は、x * w - t と 0 の比較と同じ意味になる。
# このため、xの要素に1をしたものをx'、wの要素に-tを追加したものをw'とすると、
# x' * w' と 0 の比較によって、重み付きゲートを実現できる。

# 例: x = [x0, x1, x2], w = [w0, w1, w2], t = b の場合
#     x * w と t の比較は、x * w - t と 0 の比較と同じ意味になる。
#     x * w - t = x0 * w0 + x1 * w1 + x2 * w2 - b
#               = x0 * w0 + x1 * w1 + x2 * w2 + 1 * (-b)
#     と式変形すると、x' = [x0, x1, x2, 1], w' = [w0, w1, w2, -b] とすることにより、
#     x * w - t = x' * w' となる。

# 重み付きユニットの入力と重み、しきい値から出力を求める。
def weighted_gate(x, w):
    weighted_value = np.sum(x * w)
    if (weighted_value >= 0):
        return 1
    else:
        return 0

# 重み付きユニットを使った3入力ANDゲート
def AND3(x0, x1, x2):
    x = np.asarray([x0, x1, x2, 1])
    w = np.asarray([0.2, 0.2, 0.2, -0.5])
    return weighted_gate(x, w)

signals3 = [[0, 0, 0], [1, 0, 0], [1, 1, 0], [1, 1, 1]]
for signal in signals3:
    fmt = "(x0, x1, x2) = ({0}, {1}, {2})  =>  AND3(x0, x1, x2) = {3}"
    print(fmt.format(signal[0], signal[1], signal[2],
                    AND3(signal[0], signal[1], signal[2])))

(x0, x1, x2) = (0, 0, 0)  =>  AND3(x0, x1, x2) = 0
(x0, x1, x2) = (1, 0, 0)  =>  AND3(x0, x1, x2) = 0
(x0, x1, x2) = (1, 1, 0)  =>  AND3(x0, x1, x2) = 0
(x0, x1, x2) = (1, 1, 1)  =>  AND3(x0, x1, x2) = 1


In [6]:
# XORゲート
# 2入力ゲートの入力2つのうち、一方だけが1のときにのみ1を出力し、
# それ以外の場合は0を出力する論理ゲート。
# 入力x0, x1と出力yの対応関係は次の通り。
# |x0 |x1 ||y  |
# --------------
# |  0|  0||  0|
# |  0|  1||  1|
# |  1|  0||  1|
# |  1|  1||  0|
def XOR0(x0, x1):
    if x0 == 1 and x1 == 0:
        return 1
    elif x0 == 0 and x1 == 1:
        return 1
    else:
        return 0

for signal in itertools.product([0, 1], repeat=2):
    print("(x0, x1) = ({0}, {1})  =>  XOR(x0, x1) = {2}"
          .format(signal[0], signal[1], XOR0(signal[0], signal[1])))

(x0, x1) = (0, 0)  =>  XOR(x0, x1) = 0
(x0, x1) = (0, 1)  =>  XOR(x0, x1) = 1
(x0, x1) = (1, 0)  =>  XOR(x0, x1) = 1
(x0, x1) = (1, 1)  =>  XOR(x0, x1) = 0


In [7]:
# 論理ゲートの組み合わせによるXORゲートの実現

# AND, OR, NOT, NAND, NORの各論理ゲートは、重み付きユニット1つで実現できる。
# ところが、XORゲートは、重み付きユニット1つでは実現できない。
# これは、論理ゲート1つだけでは、線形識別ができないことによる。
# 線形識別: 入力値の重み付け和によって、出力が0か1かを区別すること。

# XORゲートを重み付きユニットで実現するには、複数のユニットを組み合わせる必要がある。
# s0 = OR(x0, x1), s1 = NAND(x0, x1) とすると、x0, x1, s0, s1 の対応関係は次のようになる。
# |x0 |x1 ||s0 |s1 |
# ------------------
# |  0|  0||  0|  1|
# |  0|  1||  1|  1|
# |  1|  0||  1|  1|
# |  1|  1||  1|  0|
# 上の表から、y = AND(s0, s1) とすれば、y = XOR(x0, x1) となる。

def AND(x0, x1):
    if x0 == 0 or x1 == 0:
        return 0
    else:
        return 1

def OR(x0, x1):
    if x0 == 0 and x1 == 0:
        return 0
    else:
        return 1

def NAND(x0, x1):
    if x0 == 1 and x1 == 1:
        return 0
    else:
        return 1

def XOR(x0, x1):
    # 入力層→中間層
    s0 = OR(x0, x1)
    s1 = NAND(x0, x1)
    # 中間層→出力層
    return AND(s0, s1)

for signal in itertools.product([0, 1], repeat=2):
    print("(x0, x1) = ({0}, {1})  =>  XOR(x0, x1) = {2}"
          .format(signal[0], signal[1], XOR(signal[0], signal[1])))

(x0, x1) = (0, 0)  =>  XOR(x0, x1) = 0
(x0, x1) = (0, 1)  =>  XOR(x0, x1) = 1
(x0, x1) = (1, 0)  =>  XOR(x0, x1) = 1
(x0, x1) = (1, 1)  =>  XOR(x0, x1) = 0


In [10]:
# 重み付きユニットによるXORゲートの実現

def AND2(x0, x1):
    return weighted_gate(np.asarray([x0, x1, 1]),
                         np.asarray([0.3, 0.3, -0.5]))

def OR2(x0, x1):
    return weighted_gate(np.asarray([x0, x1, 1]),
                         np.asarray([0.3, 0.3, -0.2]))

def NAND2(x0, x1):
    return weighted_gate(np.asarray([x0, x1, 1]),
                         np.asarray([-0.3, -0.3, 0.5]))

def XOR2(x0, x1):
    # 入力層→中間層
    s0 = OR2(x0, x1)
    s1 = NAND2(x0, x1)
    # 中間層→出力層
    return AND2(s0, s1)

for signal in itertools.product([0, 1], repeat=2):
    print("(x0, x1) = ({0}, {1})  =>  XOR2(x0, x1) = {2}"
          .format(signal[0], signal[1], XOR2(signal[0], signal[1])))

(x0, x1) = (0, 0)  =>  XOR2(x0, x1) = 0
(x0, x1) = (0, 1)  =>  XOR2(x0, x1) = 1
(x0, x1) = (1, 0)  =>  XOR2(x0, x1) = 1
(x0, x1) = (1, 1)  =>  XOR2(x0, x1) = 0


In [14]:
# パーセプトロン
# 複数の信号を入力として受け取り、入力信号の値に基づいて信号を出力する仕組み。
# 重み付きユニットは、最も単純なパーセプトロンとみなせる。

# 単純パーセプトロン
# 入力層、出力層の2層で構成されるパーセプトロン。
# 重み付きユニットを用いて、入力信号から出力信号を得る。
# AND, OR, NOT, NAND, NORの論理ゲートを表現できる。

# 多層パーセプトロン
# 入力層と出力層の間に、中間層を設けたパーセプトロン。
# 入力信号から、重み付きユニットによって中間信号を得たら、
# さらに別の重み付きユニットによって、中間信号から出力信号を得る。
# XORは単純パーセプトロンで表現できないが、中間層を1つ設けると表現できる。

# 例:
# 2入力(x0, x1), 2出力(y0, y1), 中間層のユニットが2(s0, s1)のとき
# 入力層→中間層
# s0: x0 * w000 + x1 * w001 - b00 と 0 の大小を比較して求める
# s1: x0 * w010 + x1 * w011 - b01 と 0 の大小を比較して求める
# 中間層→出力層
# y0: s0 * w100 + s1 * w101 - b10 と 0 の大小を比較して求める
# y1: s0 * w110 + s1 * w111 - b11 と 0 の大小を比較して求める

# 上記で定義した weighted_gate 関数を使うと、それぞれ
# s0: x = np.asarray([x0, x1, 1]), w = np.asarray([w000, w001, -b00])
# s1: x = np.asarray([x0, x1, 1]), w = np.asarray([w010, w011, -b01])
# y0: x = np.asarray([s0, s1, 1]), w = np.asarray([w100, w101, -b10])
# y1: x = np.asarray([s0, s1, 1]), w = np.asarray([w110, w111, -b11])
# を引数として呼び出せばよい。
w000, w001, b00 = 0.3, 0.3, 0.2
w010, w011, b01 = -0.3, -0.3, -0.5
w100, w101, b10 = 0.3, 0.3, 0.5
w110, w111, b11 = -0.1, -0.5, -0.3 

w00 = np.asarray([w000, w001, -b00])
w01 = np.asarray([w010, w011, -b01])
w10 = np.asarray([w100, w101, -b10])
w11 = np.asarray([w110, w111, -b11])
fmt = "x0, x1 = ({0}, {1})  => (s0, s1) = ({2}, {3}), (y0, y1) = ({4}, {5})"

for signal in itertools.product([0, 1], repeat=2):
    x = np.asarray([signal[0], signal[1], 1])
    s0 = weighted_gate(x, w00)
    s1 = weighted_gate(x, w01)
    s = np.asarray([s0, s1, 1])
    y0 = weighted_gate(s, w10)
    y1 = weighted_gate(s, w11)
    print(fmt.format(signal[0], signal[1], s0, s1, y0, y1))

x0, x1 = (0, 0)  => (s0, s1) = (0, 1), (y0, y1) = (0, 0)
x0, x1 = (0, 1)  => (s0, s1) = (1, 1), (y0, y1) = (1, 0)
x0, x1 = (1, 0)  => (s0, s1) = (1, 1), (y0, y1) = (1, 0)
x0, x1 = (1, 1)  => (s0, s1) = (1, 0), (y0, y1) = (0, 1)
